In [1]:
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from barcode import EAN13
from barcode.writer import ImageWriter
import textwrap 
import re

csv = "../Order_Items_Export_-_2022-06-20.csv"
df = pd.read_csv(csv)
font = "../src/Kanit-Light.ttf"
def draw_multiple_line_text(image, text, font, text_color, text_start_height):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size
    y_text = text_start_height
    lines = textwrap.wrap(text, width=40)
    for line in lines:
        line_width, line_height = font.getsize(line)
        draw.text(((image_width - line_width) / 2, y_text), 
                  line, font=font, fill=text_color)
        y_text += line_height

def draw_multiple_line_text_barcode(image, text, font, text_color, text_start_height):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size
    y_text = text_start_height
    lines = textwrap.wrap(text, width=40)
    for line in lines:
        line_width, line_height = font.getsize(line)
        draw.text(((image_width - line_width) / 5, y_text), 
                  line, font=font, fill=text_color)
        y_text += line_height

def createbarcode(sku):
    num_list = []
    try:
        for c in re.findall('[a-zA-Z]+',sku)[0]: num_list.append(str(ord(c)))
    except:pass
    str_to_num = "".join(num_list) + "".join(re.findall('[0-9]+',sku))
    i = 0
    while len(str_to_num) <13:
        str_to_num = f"{str_to_num}{i}"
        i+=1
    code = EAN13(str_to_num,writer=ImageWriter())
    code.save('Barcode_Copy')
    img = Image.open('Barcode_Copy.png')
    return img

In [ ]:
#1 W8
#Barcode_Copy
c = 0
width = 400     #4cm
height = 300    #3cm
fonts = ImageFont.truetype(font, size=20)
fonts_weight = ImageFont.truetype(font, size=15)
fonts_sku = ImageFont.truetype(font, size=20)
image_list = []
df.sort_values(by=['Product Name'],inplace=True)
for index, row in df.iterrows():
    if not (row['Product SKU'][:2] == 'VB' or row['Product SKU'][:2] == 'FT'): continue
    product = row['Product Name']
    product_name = product.split('(')[0]
    product_weight = f"{re.findall('[0-9]+',product)[0]} กรัม"
    product_sku = row['Product SKU']
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    text_color = (0,0,0)   #black
    draw_multiple_line_text_barcode(img, product_name, fonts, text_color, height*(5/10))
    draw_multiple_line_text_barcode(img, product_weight, fonts_weight, text_color, height*(6/10))
    draw_multiple_line_text_barcode(img, product_sku, fonts_sku, text_color, height*(1/20))
    code = createbarcode(product_sku)
    code = code.resize((int(width/2),int(height/4)))
    img.paste(code,(int(width*(1/50)),int(height*(2/10))))
    subloop = int(row['Line Item Quantity'])
    for copy in range(subloop): image_list.append(img.convert('RGB'))

image_list[0].save('Quantity_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
#2 check
#Product_type
logo = Image.open("../src/logo-web.png")
width = 1240
height = 1754
head_font = ImageFont.truetype(font, size=160)
content_1_font = ImageFont.truetype(font, size=100)
content_2_font = ImageFont.truetype(font, size=60)
image_list = []
for index, row in df.iterrows():
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)
    text_color = (0,0,0)
    draw_multiple_line_text(img, row['Product SKU'], head_font, text_color, 400)
    draw_multiple_line_text(img, f"{re.findall('[0-9]+',row['Product Name'])[0]}  กรัม/แพ็ค", content_1_font, text_color, 700)
    draw_multiple_line_text(img, f"{row['Product Name']}     {row['Product Categories']}", content_2_font, text_color, 1000)
    draw_multiple_line_text(img, str(row['Line Item Quantity']), head_font, text_color, 1200)
    draw_multiple_line_text(img, "แพ็ค", content_1_font, text_color, 1400)
    img.paste(logo,(int(width/2.4),200))
    image_list.append(img.convert('RGB'))

image_list[0].save('Product_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
#3
filter_values = ['ผัก','ผลไม้']
newdf = df.loc[~df['Product Categories'].isin(filter_values)]
width = 800
height = 600
fonts = ImageFont.truetype(font, size=20)
image_list = []
for index, row in newdf.iterrows():
    product = row['Product Name']
    product_name = product.split('(')[0]
    product_weight = f"{re.findall('[0-9]+',product)[0]} กรัม"
    product_sku = row['Product SKU']
    textl1 = f"{product_name}   Nutrition information"
    textl2 =f"SKU : {product_sku}   Nutrtient Composition per 100 g Edible Potion"
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    text_color = (0,0,0)   #black
    draw_multiple_line_text(img, textl1, fonts, text_color, height*(1/9))
    draw_multiple_line_text(img, textl2, fonts, text_color, height*(2/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(3/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(4/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(5/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(6/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(7/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(8/9))
    # code = createbarcode(product_sku)
    # code = code.resize((int(width/3),int(height/4)))
    # img.paste(code,(int(width*(1/3)),int(height*(1/2))))
    subloop = int(row['Line Item Quantity'])
    for copy in range(subloop): image_list.append(img.convert('RGB'))

image_list[0].save('FilVeg-Fruit_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
#4 Check
#Durian_crate (5) #PDF
#sum แล้วหาร ใหญ่ 24  ,เล็ก 12 ,เดี่ยว
Durian_ID = [7576,7562,7564,8140,8216]    #กล่อง ,ลังเล็ก, ลังใหญ่, ก้านยาวกล่องเดี่ยว, ภูเขาไฟกล่องเดียว
boxshare = [24,12]          #ลังใหญ่ ,ลังเล็ก
durianDF = df.loc[df['Product ID'].isin(Durian_ID)]
sumbox = 0
width = 1240
height = 1754

fonts = ImageFont.truetype(font, size=90)
for index, row in durianDF.iterrows():
    if row['Product ID'] == Durian_ID[2]: sumbox += 24*int(row['Line Item Quantity'])
    elif row['Product ID'] == Durian_ID[1]: sumbox += 12*int(row['Line Item Quantity'])
    else: sumbox += 1*int(row['Line Item Quantity'])
box = { 'big':int(sumbox/boxshare[0]),
        'small':int((sumbox%boxshare[0])/boxshare[1]),
        'single':int((sumbox%boxshare[0])%boxshare[1])}
img = Image.new('RGB', (width, height), color='white')
imgDraw = ImageDraw.Draw(img)
text_color = (0,0,0)
text_start_height = height/5
text = [f"กล่องเดี่ยวทั้งหมด  {sumbox}  กล่อง",
        f"ลังใหญ่  {box['big']}  ลัง",
        f"ลังเล็ก  {box['small']}  ลัง",
        f"กล่องเดี่ยว  {box['single']}  ลัง"]
for t in text:
    draw_multiple_line_text(img, t, fonts, text_color, text_start_height)
    text_start_height += int(height/6)
img.save('Durian.pdf', save_all=True)
print(box)
print(durianDF)

In [ ]:
#5
#cover
width = 4*100
height = 4*75
image_list = []
fonts = ImageFont.truetype("../src/Kanit-Medium.ttf", size=90)
logo = Image.open("../src/LogoBW.png")
Max = int(input("Enter max number: "))

for i in range(Max):
    num = str(i+1)
    while len(num)<3:
        num = f"0{num}"
    img = Image.new('RGB', (width, height), color='white')
    imgDraw = ImageDraw.Draw(img)
    textWidth, textHeight = imgDraw.textsize(num, font=fonts)
    xText = (width - textWidth) / 2
    yText = (height - textHeight) / 2
    imgDraw.text((xText, yText), num, font=fonts, fill=(0, 0, 0))
    img.paste(logo,(int(width/2.7),50))
    image_list.append(img.convert('RGB'))
image_list[0].save('Amount_pages.pdf', save_all=True, append_images=image_list[1:])

In [11]:
customer_df = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
order_id = {}
for index, row in customer_df.iterrows():
    if not row['No.'] in order_id: order_id[row['No.']] = row['Customer Name']
order_id

{8421: 'Jammie Jamjam',
 8381: 'worawan',
 8380: 'Waree Sitet',
 8379: 'แต้ว อาหารไทย',
 8376: 'Nichcha',
 8375: 'Netchanok Bennett',
 8372: 'Siriyapha',
 8368: 'Judy Law',
 8367: 'Rotchana Kerin',
 8364: 'Nat',
 8362: 'Suchitra Holtom',
 8359: 'Simon'}

In [15]:
#5
#cover x3
width = 4*100
height = 4*75
image_list = []
no_fonts = ImageFont.truetype("../src/Kanit-Medium.ttf", size=90)
name_fonts = ImageFont.truetype("../src/Kanit-Medium.ttf", size=30)
logo = Image.open("../src/LogoBW.png")
customer_df = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
#Max = int(input("Enter max number: "))
order_id = {}
for index, row in customer_df.iterrows():
    if not row['No.'] in order_id: order_id[row['No.']] = row['Customer Name']
text_color = (0,0,0)
for no in order_id:
    customer_name = order_id[no]
    text_color = (0,0,0)
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)
    draw_multiple_line_text(img, str(no), no_fonts, text_color, height*(3/10))
    draw_multiple_line_text(img, customer_name, name_fonts, text_color, height*(7/10))
    img.paste(logo,(int(width/2.7),50))
    for copy in range(3): image_list.append(img.convert('RGB'))
image_list[0].save('OrderLabel_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
def createbarcode(sku):
    num_list = []
    try:
        for c in re.findall('[a-zA-Z]+',sku)[0]: num_list.append(str(ord(c)))
    except:pass
    str_to_num = "".join(num_list) + "".join(re.findall('[0-9]+',sku))
    i = 0
    while len(str_to_num) <13:
        str_to_num = f"{str_to_num}{i}"
        i+=1
    code = EAN13(str_to_num,writer=ImageWriter())
    code.save('Barcode_Copy')
    img = Image.open('Barcode_Copy.png')
    return [img,str_to_num]

def readbarcode(num):
    if num[:-1] == "697970012345":return "EOF"
    num = num[:-1]
    while num[-1] != "0": num = num[:-1]
    num = num[:-1]      #detect trash digits
    chr_list = []
    alpha = textwrap.wrap(num[:-4],2)
    for c in alpha: chr_list.append(str(chr(int(c))))
    return str("".join(chr_list)+num[-4:])
a = "VB0411"
b = "DTF1420"
c = "IF0580"
#a = "EOF"

In [ ]:
customer_df = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
def unitPrice(df):
    product_price = {}
    for index,row in df[['Item Code','Item Price']].drop_duplicates(keep='first').iterrows(): 
        product_price[row['Item Code']] = row['Item Price']
    return product_price
#unitPrice(customer_df)

In [ ]:
#new btn1 (Total USD)
customer_df = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
def unitPrice(df):
    product_price = {}
    for index,row in df[['Item Code','Item Price']].drop_duplicates(keep='first').iterrows(): 
        product_price[row['Item Code']] = row['Item Price']
    return product_price

product_price = unitPrice(customer_df)
use_df = df[['Product SKU','Product Name','Line Item Quantity','Product Categories']]
for index, row in use_df.iterrows():    #calculate weight
    net_weight = str((int(re.findall('[0-9]+',row['Product Name'])[0])/1000)*int(row['Line Item Quantity']))  #Kg
    if len(net_weight.split('.')[-1]) >1:
        #net_weight = "".join(net_weight.split('.')[0]+'.'+net_weight.split('.')[-1][0])
        net_weight = format(float(net_weight),'.2f')
    price = product_price[row['Product SKU']] 
    product_name = row['Product Name'].split('(')
    if len(product_name) > 2:
        product_name.pop()
        product_name = "".join(product_name)[:-2]
    else:
        product_name = product_name[0][:-1]
    use_df.loc[index,'Product Name'] = str(product_name)
    use_df.loc[index,'N.W. (kg)'] = float(net_weight)
    use_df.loc[index,'Unit Price (USD)'] = price     #edit
    use_df.loc[index,'Total (USD)'] = format(float(net_weight)*price,'.2f')

categories = list(set(use_df['Product Categories'].values.tolist()))
df_dict = {}
for product in categories:
    new_df = use_df.loc[use_df['Product Categories']==product][['Product SKU','Product Name','N.W. (kg)','Unit Price (USD)','Total (USD)']].sort_values('Product SKU')
    new_df.rename(columns={'Product SKU':'Code'},inplace=True)
    numlist = []
    for n in range(len(new_df.index)): numlist.append(n+1)
    new_df.insert(0,'No',numlist)
    df_dict[product] = new_df.values.tolist()
df_dict

In [ ]:
#new btn2 (Packing)
# use_df = df[['Product SKU','Product Name','Line Item Quantity','Product Categories']]
# for index, row in use_df.iterrows():    #calculate weight
#     net_weight = str((int(re.findall('[0-9]+',row['Product Name'])[0])/1000)*int(row['Line Item Quantity']))  #Kg
#     if len(net_weight.split('.')[-1]) >1:
#         net_weight = "".join(net_weight.split('.')[0]+'.'+net_weight.split('.')[-1][0])
#     use_df.loc[index,'Item'] = f"{row['Product SKU']} {row['Product Name'].split('(')[0]}"
#     use_df.loc[index,'Packing'] = 'xxx'
#     use_df.loc[index,'Net WT (Kg)'] = float(net_weight)
#     use_df.loc[index,'Gross WT (Kg)'] = 1
#     use_df.loc[index,'Volume (x1000 cc.)'] = float(net_weight)*2

# categories = list(set(use_df['Product Categories'].values.tolist()))
# df_dict = {}
# for product in categories:
#     new_df = use_df.loc[use_df['Product Categories']==product].sort_values('Product SKU')
#     new_df.rename(columns={'Line Item Quantity':'Cts'},inplace=True)
#     numlist = []
#     for n in range(len(new_df.index)): numlist.append(n+1)
#     new_df.insert(0,'No',numlist)
#     new_df.drop(['Product SKU','Product Name','Product Categories'],axis=1,inplace=True)
#     cst_column = new_df.pop('Cts')
#     new_df.insert(3,'Cts',cst_column)
#     df_dict[product] = new_df
#df_dict

In [ ]:
re.findall('[a-zA-Z]+','FT0011')[0]

In [ ]:
#new btn3 (Unit)
use_df = df[['Product SKU','Product Name','Line Item Quantity','Product Categories']]
product_unit = {'FT0011':'12Box','FT0012':'24Box','VB':'Pack','DFT':'Piece','FT':'Kg','Else':'Pack'}
for index, row in use_df.iterrows():    #calculate weight
    net_weight = str((int(re.findall('[0-9]+',row['Product Name'])[0])/1000)*int(row['Line Item Quantity']))  #Kg
    if len(net_weight.split('.')[-1]) >1:
        #net_weight = "".join(net_weight.split('.')[0]+'.'+net_weight.split('.')[-1][0])
        net_weight = format(float(net_weight),'.2f')
    product_name = row['Product Name'].split('(')
    if len(product_name) > 2:
        product_name.pop()
        product_name = "".join(product_name)[:-2]
    else:
        product_name = product_name[0][:-1]
    use_df.loc[index,'Product Name'] = str(product_name)
    
    if row['Product SKU'] in product_unit:
        use_df.loc[index,'Unit'] = product_unit[row['Product SKU']]
    elif re.findall('[a-zA-Z]+',row['Product SKU'])[0] in product_unit:
        use_df.loc[index,'Unit'] = product_unit[re.findall('[a-zA-Z]+',row['Product SKU'])[0]]
    else:
        use_df.loc[index,'Unit'] = product_unit['Else']
    use_df.loc[index,'N.W. (kg)'] = float(net_weight)

categories = list(set(use_df['Product Categories'].values.tolist()))
df_dict = {}
for product in categories:
    new_df = use_df.loc[use_df['Product Categories']==product][['Product SKU','Product Name','Line Item Quantity','Unit','N.W. (kg)']].sort_values('Product SKU')
    new_df.rename(columns={'Product SKU':'Code','Line Item Quantity':'Quantity'},inplace=True)
    numlist = []
    for n in range(len(new_df.index)): numlist.append(n+1)
    new_df.insert(0,'No',numlist)
    df_dict[product] = new_df.values.tolist()


In [ ]:
l = []
for n in range(20): l.append(n+1)
l


In [ ]:
print(b)
print(createbarcode(b)[1])
print(readbarcode(createbarcode(b)[1]))

In [ ]:
x = createbarcode(c)[1]
print(x)
readbarcode(x)

In [ ]:
createbarcode(b)[1]

In [ ]:
createbarcode(a)[0]

In [ ]:
#7
newdf = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
order_id = set(newdf['No.'].tolist())
fonts = ImageFont.truetype(font, size=120)
image_list = []
text_color = (0,0,0)   #black
width = 1240  
height = 1754 
for order in order_id:
    barcode_order = createbarcode(str(order)).resize((int(width/1.5),int(height/7)))
    barcode_EOF = createbarcode("EOF").resize((int(width/3),int(height/9)))
    barcode_zero = createbarcode("0").resize((int(width/3),int(height/9)))
    customer_name = set(newdf.loc[newdf['No.']==order]['Customer Name'].tolist()).pop()
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    draw_multiple_line_text(img, f"No.  {str(order)}", fonts, text_color, height*(4/10))
    draw_multiple_line_text(img, customer_name, fonts, text_color, height*(5.5/10))
    img.paste(barcode_order,(int(width*(1/6)),int(height*(2/10))))
    img.paste(barcode_EOF,(int(width*(1/6)),int(height*(8/10))))
    img.paste(barcode_zero,(int(width*(3/6)),int(height*(8/10))))
    image_list.append(img.convert('RGB'))
image_list[0].save('Customer_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
#newdurian
Durian_ID = [7576,7562,7564,8140,8216]    #กล่อง ,ลังเล็ก, ลังใหญ่, ก้านยาวกล่องเดี่ยว, ภูเขาไฟกล่องเดียว
Durian_SKU = ['FT0012','FT0011','FT0015','FT0035','FT0025'] #ลังใหญ่, ลังเล็ก, กล่อง, ภูเขาไฟกล่องเดียว, ก้านยาวกล่องเดี่ยว
boxshare = [24,12]          #ลังใหญ่ ,ลังเล็ก
durian_product_DF = df.loc[df['Product SKU'].isin(Durian_SKU)]
sumbox = 0
durian_product_DF

In [ ]:
Customer_df = pd.read_csv('../../orders-2022-06-20-00-16-07.csv')
Durian_SKU = ['FT0012','FT0011','FT0015','FT0035','FT0025']     #ลังใหญ่, ลังเล็ก, กล่อง, ภูเขาไฟกล่องเดียว, ก้านยาวกล่องเดี่ยว
Customer_durian_df = Customer_df.loc[Customer_df['Item Code'].isin(Durian_SKU)][['No.','Customer Name','Item Code','Item Name','Item Qty']]
for index, row in Customer_durian_df.iterrows():
    if row['Item Code'] == 'FT0012': Customer_durian_df.loc[index,'total'] = str(24*int(row['Item Qty']))
    elif row['Item Code'] == 'FT0011': Customer_durian_df.loc[index,'total'] = str(12*int(row['Item Qty']))
    else: Customer_durian_df.loc[index,'total'] = str(int(row['Item Qty']))
df_list = []
for item_code in Durian_SKU:
    df_list.append(Customer_durian_df.loc[Customer_durian_df['Item Code'].isin([item_code])])
df_list